### Demonstration of GPU Accelerated SigMF Reader

In [1]:
import json
import numpy as np
import cupy as cp
import cusignal

cusignal.precompile_kernels()

In [2]:
meta_file = '/data/oracle/KRI-16Devices-RawData/2ft/WiFi_air_X310_3123D7B_2ft_run1.sigmf-meta'
data_file = '/data/oracle/KRI-16Devices-RawData/2ft/WiFi_air_X310_3123D7B_2ft_run1.sigmf-data'

### Baseline Reader (CPU, Numpy)

In [3]:
with open(meta_file, 'r') as f:
    md = json.loads(f.read())

if md['_metadata']['global']['core:datatype'] == 'cf32':
    data_type = np.complex64

In [4]:
%%timeit
data_cpu = np.fromfile(data_file, dtype=data_type)

137 ms ± 334 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Baseline Reader (GPU, Numpy)

In [5]:
%%timeit
data_gpu = cp.fromfile(data_file, dtype=data_type)
cp.cuda.runtime.deviceSynchronize()

198 ms ± 2.06 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### cuSignal - Default Reader

In [6]:
%%timeit
data_cusignal = cusignal.read_sigmf(data_file, meta_file)
cp.cuda.runtime.deviceSynchronize()

82 ms ± 284 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### cuSignal - Use Pinned GPU Array

In [7]:
binary = cusignal.read_bin(data_file)
buffer = cusignal.get_pinned_mem(binary.shape, cp.ubyte)

In [8]:
%%timeit
data_cusignal_pinned = cusignal.read_sigmf(data_file, meta_file, buffer)
cp.cuda.runtime.deviceSynchronize()

82.1 ms ± 66.2 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### cuSignal - Use Shared Array (Zero Copy between CPU <-> GPU)

In [9]:
binary = cusignal.read_bin(data_file)
buffer = cusignal.get_shared_mem(binary.shape, cp.ubyte)

In [10]:
%%timeit
data_cusignal_shared = cusignal.read_sigmf(data_file, meta_file, buffer)
cp.cuda.runtime.deviceSynchronize()

139 ms ± 861 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
